(sec:linreg)=
## Anwendung: Lineare Regressionsprobleme
Lineare Regressionsprobleme lassen sich auf quadratische Optimierungsprobleme zurückführen. Wir betrachten in diesem Abschnitt folgendes Problem: Es soll die geschmackliche Qualität eines Weines (basierend auf Expertenbewertungen) anhand der folgenden elf Faktoren vorhergesagt werden:

- nicht-flüchtiger Säuregehalt,
- flüchtiger Säuregehalt,
- Zitronensäure,
- Restzucker,
- Chloride,
- freies Schwefeldioxid,
- gesamtes Schwefeldioxid,
- Dichte,
- pH-Wert,
- Sulfate und
- Alkoholgehalt

Dazu wurden in {cite}`cortez_modeling_2009` Daten für 1599 portugiesische Vinho Verde Rotweinsorten beschrieben.
Wir werfen einen ersten Blick auf die Daten:

In [12]:
import pandas as pd
df_raw = pd.read_csv("Daten/winequality-red.csv", delimiter = ";" )
df_raw

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


Wir löschen zunächst doppelte Einträge und skalieren alle Features in vergleichbare Größenordnungen.

In [14]:
df = df_raw.drop_duplicates().reset_index(drop=True)
# Spaltenweise Standardisierung: Subtraktion von Mittelwert und Division durch Standardabweichung
df = (df - df.mean()) / df.std()

df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,-0.524238,0.931657,-1.392745,-0.460987,-0.245532,-0.468381,-0.383908,0.583788,1.291397,-0.578348,-0.954023,-0.756762
1,-0.293955,1.915095,-1.392745,0.056644,0.200020,0.871682,0.603851,0.048719,-0.708135,0.124776,-0.584360,-0.756762
2,-0.293955,1.259470,-1.188180,-0.165198,0.078506,-0.085506,0.214734,0.155733,-0.321129,-0.051005,-0.584360,-0.756762
3,1.663455,-1.363032,1.471170,-0.460987,-0.265785,0.105932,0.394326,0.690802,-0.966139,-0.461161,-0.584360,0.457452
4,-0.524238,0.713115,-1.392745,-0.534935,-0.265785,-0.276944,-0.204316,0.583788,1.291397,-0.578348,-0.954023,-0.756762
...,...,...,...,...,...,...,...,...,...,...,...,...
1354,-0.869663,0.494574,-0.983615,-0.460987,-0.407552,1.158838,-0.264180,-0.106451,0.710888,0.945087,-0.861607,0.457452
1355,-1.215088,0.385303,-0.983615,-0.387040,0.038001,1.541713,-0.084587,-0.967912,0.904391,-0.461161,0.062551,-0.756762
1356,-1.387801,0.112125,-0.881332,-0.239145,-0.529066,2.211745,0.124937,-0.850197,1.355898,0.593525,0.709462,0.457452
1357,-1.387801,0.631162,-0.779049,-0.387040,-0.265785,1.541713,-0.084587,-0.662923,1.678403,0.300557,-0.214696,-0.756762


### Multivariate lineare Regression
Wir möchten nun aus den ersten elf Spalte die Zielvariable `quality` vorhersagen. Unser lineares Regressionsmodell lautet also:

\begin{align*}
f(\v x)=w_0 + w_1 x_1 + w_2 x_2 + \cdots +w_{11} x_{11} = w_0+\sum_{j=1}^{11} w_jx_j,
\end{align*}

wobei $x_1$ für das Feature `fixed acidity`, $x_2$ für das Feature `volatile acidity`, $\dots$ und $x_{11}$ für das Feature `alcohol` steht. Für diese Funktion möchten wir nun die plausibelsten Parameter $w_0, w_1, \dots, w_{11}$ identifizieren. Dies tun wir mit Hilfe des mittleren quadratischen Fehlers zwischen Vorhersage und tatsächlichem Wert (dem sog. *Label*), den wir für den gesamten Trainingsdatensatz wie folgt berechnen können:

\begin{align*}
L(w_0, w_1, \dots, w_{11}) = \frac{1}{N}\sum_{i=1}^N \left(y_i-\left(w_0 +\sum_{j=1}^{11} w_jx_{i,j}\right) \right)^2,
\end{align*}
wobei $x_{i,j}$ für den Wert des $j$-ten Features für den $i$-ten Wein steht und $N=1359$ die Anzahl der Datenpunkte (Weine) bezeichnet.

Die Minimierung von $L$ wird auch die *Methode der kleinsten Quadrate* genannt und besteht aus dem quadratischen Optimierungsproblem

```{math}
:label: eq:least-squares

\begin{align}
\min_{w_0, w_1, \dots, w_{11}} \frac{1}{N}\sum_{i=1}^N (y_i-(w_0 +\sum_{j=1}^{11} w_jx_{i,j}) )^2.
\end{align}
```

Zugegeben, diese Funktion sieht auf den ersten Blick ganz anders aus als das quadratische Optimierungsproblem in Standardform {eq}`eq:qp-standard`. Wir gehen deshalb wie folgt vor:

1. Überführen des Regressionsproblems in Standardform.
2. Aufstellen der Optimalitätsbedingungen.
3. Berechnung der Lösung mit Python.

#### Überführen in QP Standardform
Wir ordnen alle Beobachtungen in der *Designmatrix* an:

\begin{align*}
\m X=\bmat 
    1 & x_{1,1} & \dots & x_{1,11} \\
    \vdots & & & \vdots\\
    1 & x_{N,1} & \dots & x_{N,11} \emat \in \R^{N\times 12}
\end{align*}

eine Zeile der Designmatrix enthält die Features für einen Wein. Eine Spalte enthält ein Feature für den gesamten Datensatz. Weiterhin definieren wir die Koeffizientenvektor

\begin{align*}
\v w = \bmat w_0 \\ w_1 \\ \vdots \\ w_{11} \emat \in \R^{12}
\end{align*}

und den Vektor der echten Werte 
\begin{align*}
\v y = \bmat y_1 \\ y_2 \\ \vdots \\ y_{N} \emat \in \R^{N}.
\end{align*}

Wenn wir nun das Matrix-Vektor Produkt $\m X \v w$ bilden, erhalten wir den Vektor der Vorhersagen (für den gesamten Datensatz):
\begin{align*}
\m X \v w = \bmat 
            w_0 + w_1 x_{1,1} + w_2 x_{1,2} + \cdots + w_{11}x_{1,11} \\ 
            \vdots \\ 
            w_0 + w_1 x_{N,1} + w_2 x_{N,2} + \cdots + w_{11}x_{N,11} \emat \in \R^{N}
\end{align*}

Diese können wir von dem Label-Vektor $\v y$ subtrahieren, um den Vektor der Residuen (=Vorhersagefehler) zu erhalten:

\begin{align*}
\v y - \m X \v w = \bmat 
            y_1 - (w_0 + w_1 x_{1,1} + w_2 x_{1,2} + \cdots + w_{11}x_{1,11}) \\ 
            \vdots \\ 
            y_N - (w_0 + w_1 x_{N,1} + w_2 x_{N,2} + \cdots + w_{11}x_{N,11}) \emat \in \R^{N}
\end{align*}

Die Zielfunktion ist der mittlere quadratische Fehler. Diese bekommen wir, indem wir die Norm des Vektors $\v y - \m X \v w$ berechnen:

\begin{align*}
\frac{1}{N}\norm{\v y - \m X \v w}_2^2  &= (\v y - \m X \v w)^T(\v y - \m X \v w) \\&= (y_1 - (w_0 + w_1 x_{1,1} + w_2 x_{1,2} + \cdots + w_{11}x_{11}))^2 + \cdots + (y_N - (w_0 + w_1 x_{N,1} + w_2 x_{N,2} + \cdots + w_{11}x_{11}))^2
\end{align*}

Wir halten fest: Das Problem der kleinsten Quadrate {eq}`eq:least-squares` ist äquivalent zu folgendem Problem:

\begin{align*}
\min_{\v w} \frac{1}{N}(\v y - \m X \v w)^T(\v y - \m X \v w).
\end{align*}

Um dem Problem in Standardform noch ein wenig näher zu kommen, halten wir fest, dass die Multiplikation der Zielfunktion mit einer positiven Zahl zwar den Zielfunktionswert am Minimalpunkt verändert, nicht aber das Minimum selbst. Wir können deshalb statt dem Faktor $\frac{1}{N}$ den Faktor $\frac{1}{2}$ benutzen (warum, sehen wir gleich):

\begin{align*}
\min_{\v w} \frac{1}{2}(\v y - \m X \v w)^T(\v y - \m X \v w).
\end{align*}

Nun multiplizieren wir die Terme aus unter Berücksichtigung der Rechengesetze für die Matrixmultiplikation:

\begin{align*}
\frac{1}{2}(\v y - \m X \v w)^T(\v y - \m X \v w) &= \frac{1}{2}(\v y^T\v y - \v y^T \m X\v w - \underbrace{(\m X\v w)^T}_{=\v y^T \m X\v w} \v y + (\m X \v w)^T (\m X \v w))\\
&=\frac{1}{2}\v w^T \m X^T \m X \v w -\v y^T\m X \v w + \frac{1}{2}\v y^T\v y
\end{align*}

Damit haben wir gezeigt: Training einer multivariaten linearen Regression ist ein quadratisches Optimierungsproblem der Form $\frac{1}{2}\v w^T\v A\v w+\v b^T\v w+c$ mit den Problemdaten

\begin{align*}
\m A &= \m X^T \m X\\
\v b &= (-\v y^T\m X)^T = - \m X^T \v y\\
c & = \frac{1}{2}\v y^T\v y
\end{align*}


#### Aufstellen der Optimalitätsbedingungen
Nun können wir auf {prf:ref}`thm:krit-qp` zurückgreifen, um das Minimum zu bestimmen. Dafür vergewissern wir uns zunächst, dass die Funktion konvex ist, die Hessematrix also positiv semi-definit ist. Für positive semi-Definitheit muss für jeden Vektor $\v w\neq \v 0$ gelten

\begin{align*}
\v w^T \m X^T \m X \v w \geq 0
\end{align*}

Dies ist der Fall, da
\begin{align*}
\v w^T \m X^T \m X \v w = (\m X \v w)^T \m X \v w = \norm{\m X \v w}_2^2 \geq 0.
\end{align*}
Kritische Punkte sind also in jedem Fall Minima.

Wir bestimmen nun die kritischen Punkte. Nach {prf:ref}`thm:krit-qp` erhält man sie durch Lösen des Gleichungssystems

\begin{align*}
\m X^T \m X \v w=\m X^T \v y.
\end{align*}

Man nennt diese Gleichungen auch *Normalengleichungen*.

Weiterhin gilt: Falls $\m X^T \m X $ invertierbar ist, so ist der einzige kritische Punkt $\v w^*$
\begin{align*}
\v w^*=(\m X^T \m X)^{-1}\m X^T \v y
\end{align*}

Wann ist $\m X^T \m X$ nicht invertierbar? 
% TODO: Rangüberlegungen auf X (statt Gram). 
$\m X^T \m X$ ist ein Beispiel für eine [Gram'sche Matrix](https://en.wikipedia.org/wiki/Gram_matrix). Diese haben die Eigenschaft, dass sie genau dann invertierbar sind, wenn alle Spalten von $\m X$ linear unabhängig sind. In der Sprache der linearen Regression ausgedrückt: keines der Features kann durch Linearkombination der anderen Features ausgedrückt werden. Falls die Features linear abhängig sind, ist die Matrix $\m X^T \m X$ nicht invertierbar, sondern es gibt unendlich viele Lösungen der Normalengleichungen. Dabei sind alle Lösungen gleichwertig, liefern also den gleichen Zielfunktionswert.

Eine Folgerung daraus ist, dass $\m X^T \m X$ nicht invertierbar ist, wenn es mehr Features als Trainingsdatenpunkte gibt, also $\m X$ mehr Spalten als Zeilen besitzt (dann sind die Spalten nämlich automatisch linear abhängig).


#### Berechnung der Lösung mit Python
Wir berechnen nun die Lösung des Regressionsproblems mit Methoden der linearen Algebra, die im Paket `numpy` bzw. im Submodul `numpy.linalg` implementiert sind.

In [38]:
import numpy as np

# Designmatrix
X = df.drop("quality", axis=1)
# Spalte mit Einsen für w_0 Koeffizienten
X.insert(0, "ones", 1)

# Labelvektor
y = df["quality"]

A = X.T @ X
b = -X.T @ y

# Lösen der Normalengleichungen
w = np.linalg.solve(A, -b)

print("Gewichte der Features:")
for i in range(1,12):
    print(f"{X.columns[i].ljust(20)}: {w[i]: .3f}")
    


Gewichte der Features:
fixed acidity       :  0.027
volatile acidity    : -0.249
citric acid         : -0.039
residual sugar      :  0.012
chlorides           : -0.116
free sulfur dioxide :  0.042
total sulfur dioxide: -0.110
density             : -0.020
pH                  : -0.086
sulphates           :  0.190
alcohol             :  0.380


Das legt nahe, dass Sulfat- und Alkoholgehalt einen positiven Einfluss auf die Weinqualität haben.

### Ridge Regression

Wir betrachten als nächstes eine Erweiterung des Modells: Die Ridge Regression. Dies ist eine sog. *Regularisierungstechnik*. Sie führt zum einen dazu, dass die Koeffizienten betragskleiner werden, die Vorhersagegüte allerdings ein schlechter. Dafür erhält man ein Modell, welches weniger stark von Schwankungen in den Daten abhängt, in der Regel also eine statistisch zuverlässigere Vorhersage liefert. 

Bei der Ridge Regression wird ein Strafterm zur Zielfunktion addiert, der betragsgroße Koeffizienten "bestraft":

\begin{align*}
\min_{\v w} L(\v w) = \frac{1}{2}(\v y - \m X \v w)^T(\v y - \m X \v w) + \lambda \sum_{j=0}^{11} w_j.
\end{align*}

Mit einer vorher fest gewählten Zahl $\lambda > 0$.
Das Ziel der Optimierung ist nun nicht mehr nur den quadratischen Fehler $(\v y - \m X \v w)^T(\v y - \m X \v w)$ zu minimieren, sondern auch die Koeffizienten $w_j$ möglichst betragsklein zu wählen. Die Wichtigkeit dieser beiden Ziele wird durch den Hyperparameter $\lambda$ ausbalanciert.

Als nächstes bringen wir das Optimierungsproblem wieder in Standardform und lösen es danach in Python. Dafür stellen wir zunächst fest, dass wir den Strafterm mit Hilfe der Einheitsmatrix $\I$ wie folgt schreiben können:
\begin{align*}
\lambda \sum_{j=0}^{11} w_j &= \lambda \v w^T \v w \\
                &= \v w^T \bmat \lambda & & \\ & \ddots & \\ && \lambda\emat  \v w \\
                &=\v w^T \lambda \I \v w 
\end{align*}

Wir multiplizieren den Term $\frac{1}{2}(\v y - \m X \v w)^T(\v y - \m X \v w)$ wieder aus und schreiben die Zielfunktion $L(\v w)$ als
\begin{align*}
L(\v w) &= \frac{1}{2}\v w^T \m X^T \m X \v w -\v y^T\m X \v w + \frac{1}{2}\v y^T\v y + \v w^T \lambda \I \v w 
\end{align*}
Zusammenfassen der Terme ergibt

\begin{align*}
L(\v w)= \frac{1}{2}\v w^T (\m X^T \m X + \lambda \I) \v w -\v y^T\m X \v w + \frac{1}{2}\v y^T\v y
\end{align*}

Damit haben wir die Ridge Regression auf QP-Standardform gebracht, indem wir als Hessematrix $\m A:=\m X^T \m X +\lambda \I$ gewählt haben.

Analog zur multivariaten linearen Regression lauten die Normalengleichungen dafür:
\begin{align*}
(\m X^T \m X + \lambda \I) \v w=\m X^T \v y
\end{align*}
und das Minimum $\v w^*$ ist
\begin{align*}
\v w^*=(\m X^T \m X + \lambda \I)^{-1}\m X^T \v y
\end{align*}

Interessant: Die Hessematrix der Ridge Regression, $\m X^T \m X +\lambda \I$, ist immer invertierbar. Der Term $\lambda \m I$ macht die Matrix $\m X^T \m X$ *regulär* (d.h. invertierbar). Daher auch der Name Regularisierung.

Wir berechnen nun die Koeffizienten der Ridge Regression für das Rotwein Beispiel für $\lambda=100$. Achtung: für jeden Wert von $\lambda$ erhält man leicht unterschiedliche Koeffizienten. 

In [61]:
# Variablen X, y und b sind unverändert

# Hessematrix der Ridge Regression
lam = 100
A_ridge = X.T @ X + lam * np.eye(12)

# Lösen der Normalengleichungen für die Ridge Regression
w_ridge = np.linalg.solve(A_ridge, -b)

print("Gewichte der Features:")
for i in range(1,12):
    print(f"{X.columns[i].ljust(20)}: {w_ridge[i]: .3f}")

Gewichte der Features:
fixed acidity       :  0.053
volatile acidity    : -0.228
citric acid         : -0.008
residual sugar      :  0.022
chlorides           : -0.109
free sulfur dioxide :  0.034
total sulfur dioxide: -0.103
density             : -0.062
pH                  : -0.055
sulphates           :  0.181
alcohol             :  0.333
